In [ ]:
! pip uninstall -y transformer_lens
! pip install git+https://github.com/taufeeque9/TransformerLens/
! pip install git+https://github.com/minyoungg/vqtorch/
! pip install termcolor
! pip install -U accelerate
! pip install -U kaleido

In [1]:
from termcolor import colored
import plotly.express as px
import transformers
import codebook_features
import torch
import evaluate
import numpy as np
import copy
import wandb
import json
import transformer_lens.utils as utils
from collections import namedtuple
from functools import partial
from torch.nn import functional as F
import itertools
from tqdm import tqdm
import math
from datetime import datetime
import pickle
import glob
import pandas as pd


from transformers import (
    GPTNeoXConfig,
    GPTNeoXForCausalLM,
    GPT2TokenizerFast,
    pipeline,
    set_seed,
)
from torch.utils.data import IterableDataset
from codebook_features import models, run_clm, train_toy_model, trainer as cb_trainer   
from codebook_features.utils import *
from codebook_features.toy_utils import *
from codebook_features.code_search_utils import *
import os

torch.set_grad_enabled(False)

/data/codebook-features/codebook_features/train_toy_model.py:400: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="config", config_name="toy_main")


In [2]:
# cache_path = "/shared/cb_eval_acts/" + "pythia410_preproj_attn" + "_*"
# dirs = glob.glob(cache_path)
# dirs.sort(key=os.path.getmtime)
# cache_path = dirs[-1] + "/"

# tokens_str = np.load(cache_path + "tokens_str.npy")
# tokens_text = np.load(cache_path + "tokens_text.npy")
# token_byte_pos = np.load(cache_path + "token_byte_pos.npy")


In [2]:
# cb_at, ccb = "attn", "_ccb"
# model_id = "pythia70"
# model_name_or_path = "EleutherAI/pythia-70m-deduped"
# # pretrained_path = "/shared/pre_proj_comp_cb_wiki_pythia/" 
# pretrained_path = "/shared/apr14_pythia70_wiki_attention_comp_interpretable_model/"

cb_at, ccb = "preproj_attn", "_ccb"
model_id = "tinystories-21m-1l"
model_name_or_path = "roneneldan/TinyStories-1Layer-21M"
# pretrained_path = "/shared/outputs/2023-08-15/11-00-17/output_main/" # https://wandb.ai/interpretability/gpt2-codebook/runs/7m8y1bl7
pretrained_path = "/shared/outputs/2023-08-17/03-43-18/output_main/" # https://wandb.ai/interpretability/gpt2-codebook/runs/k20go6pg

# cb_at, ccb = "mlp", ""
# model_id = "pythia70"
# model_name_or_path = "EleutherAI/pythia-70m-deduped"
# pretrained_path = "/shared/outputs/2023-04-25/00-10-18/output_main/" # 70m mlp

# cb_at, ccb = "preproj_attn", "_ccb"
# model_id = "pythia410"
# model_name_or_path = "EleutherAI/pythia-410m-deduped"
# pretrained_path = "/shared/outputs/2023-04-24/14-05-06/output_main/" # 410m preproj-attn

is_attn = "attn" in cb_at
device = "cuda"
orig_cb_model = models.wrap_codebook(model_or_path=model_name_or_path, pretrained_path=pretrained_path)
orig_cb_model = orig_cb_model.to(device).eval()
orig_cb_model.disable_logging()
print("Loaded original cb model.")
hooked_kwargs = dict(center_unembed=False,fold_value_biases=False,center_writing_weights=False,fold_ln=False,refactor_factored_attn_matrices=False,device=device)

cb_model = models.convert_to_hooked_model(model_name_or_path, orig_cb_model, hooked_kwargs=hooked_kwargs)
# # cb_model = cb_model.model
cb_model.disable_logging()
cb_model = cb_model.to(device).eval()
model = orig_cb_model
tokenizer = cb_model.tokenizer

Loaded original cb model.


Using pad_token, but it is not set yet.


Loaded pretrained model roneneldan/TinyStories-1Layer-21M into HookedTransformer


In [3]:
def compare_input_on_hooked_model(input, orig_model, hooked_model):
    """Compare input on original and hooked model."""
    if isinstance(input, str):
        input = hooked_model.tokenizer(input, return_tensors="pt")["input_ids"]
    if torch.cuda.is_available():
        orig_model = orig_model.cuda()
        hooked_model = hooked_model.cuda()
        input = input.cuda()

    orig_output = orig_model(input)["logits"]
    hooked_output = hooked_model(input)
    return orig_output, hooked_output
    # assert torch.allclose(orig_output, hooked_output)

input = "Once upon a time there lived a young man named Lucifer."
a, b = compare_input_on_hooked_model(input, orig_cb_model, cb_model)
a_pred, b_pred = torch.argmax(a, dim=-1), torch.argmax(b, dim=-1) 
print(torch.allclose(a, b))


False


In [4]:
print(a_pred.shape)
print(a_pred)
print(b_pred)

torch.Size([1, 13])
tensor([[ 253,  253,  346,  253, 1157,  275,  346, 1157,  686,  346,  466, 1157,
          346]], device='cuda:0')
tensor([[ 253,  253,  346,  253, 1157,  275,  346, 1157,  686,  346,  466, 1157,
          346]], device='cuda:0')


## Code Search Cache

In [ ]:
report_to = "none"
# report_to = "all"
training_args = run_clm.TrainingArguments(
    #     no_cuda=True,
    output_dir=f"/shared/output_{cb_at}_{model_id}/",
    do_eval=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
)

model_args = run_clm.ModelArguments(model_name_or_path=pretrained_path, cache_dir="/shared/.cache/huggingface/")
if "tinystories" in model_id:
    data_args = run_clm.DataTrainingArguments(
        dataset_name="roneneldan/TinyStories", dataset_config_name=None, streaming=False, block_size=512,
    )
else:
    data_args = run_clm.DataTrainingArguments(
        dataset_name="wikitext", dataset_config_name="wikitext-103-v1", streaming=False,
    )

trainer, lm_datasets, raw_datasets, last_checkpoint = run_clm.get_trainer_and_dataset(
    model_args,
    data_args,
    training_args,
    model,
    optimizers=(None, None),
)

In [5]:
seed = 42
rng = np.random.default_rng(seed)
max_samples = 2000
indices = rng.choice(len(lm_datasets["train"]), max_samples, replace=False)
dataset = lm_datasets["train"].select(indices)
tokens = dataset["input_ids"]
tokens = np.array(tokens)
tokens_text = tokenizer.batch_decode(tokens, clean_up_tokenization_spaces=False)


In [6]:
from datetime import datetime
import pickle
import glob

trainer.args.report_on = "none"
cb_acts = {}
ft_tkns = {}

def store_cb_activations(key, codebook_ids, codebook_acts=cb_acts):
    assert len(codebook_ids.shape) == 3  # (bs, seq_len, k_codebook)
    if key not in codebook_acts:
        codebook_acts[key] = []
    codebook_acts[key].append(codebook_ids)


output_dir = f"/shared/cb_eval_acts/{model_id}_{cb_at}*"
dirs = glob.glob(output_dir)
dirs.sort(key=os.path.getmtime)
if len(dirs) > 0 and False:
    print("Loading activations from", dirs[-1])
    with open(f"{dirs[-1]}/cb_acts.pkl", "rb") as f:
        cb_acts = pickle.load(f)
    tokens = np.load(f"{dirs[-1]}/tokens.npy")
    metrics = np.load(f"{dirs[-1]}/metrics.npy", allow_pickle=True)
else:
    orig_cb_model.set_hook_fn(store_cb_activations)
    orig_cb_model.reset_codebook_metrics(), orig_cb_model.reset_hook_kwargs()
    orig_cb_model.enable_logging()
    # orig_cb_model.disable_logging()
    metrics = trainer.evaluate(dataset)
    for k, v in cb_acts.items():
        cb_acts[k] = np.concatenate(v, axis=0)
    output_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"/shared/cb_eval_acts/{model_id}_{cb_at}_" + output_dir
    os.makedirs(output_dir, exist_ok=True)
    np.save(f"{output_dir}/tokens.npy", tokens)
    np.save(f"{output_dir}/metrics.npy", metrics)
    with open(f"{output_dir}/cb_acts.pkl", "wb") as f:
        pickle.dump(cb_acts, f)
    print("Saved activations to", output_dir)

print(metrics)
num_codes = 10_000

[INFO|trainer.py:3165] 2023-08-18 12:15:34,248 >> ***** Running Evaluation *****
[INFO|trainer.py:3167] 2023-08-18 12:15:34,249 >>   Num examples = 2000
[INFO|trainer.py:3170] 2023-08-18 12:15:34,249 >>   Batch size = 64


[INFO|integrations.py:720] 2023-08-18 12:17:29,218 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: taufeeque-iit (interpretability). Use `wandb login --relogin` to force relogin


Saved activations to /shared/cb_eval_acts/tinystories-21m-1l_preproj_attn_2023-08-18_12-17-36
{'eval_loss': 1.69590425491333, 'eval_accuracy': 0.5708131115459882, 'eval_runtime': 114.968, 'eval_samples_per_second': 17.396, 'eval_steps_per_second': 0.278, 'eval_multicode_k': 1, 'eval_dead_code_fraction/layer0': 0.0001, 'eval_MSE/layer0': 4774.894357421874, 'eval_input_norm/layer0': 0.7283004955947401, 'eval_output_norm/layer0': 68.33534768486024}


In [6]:
tokens_str = [tokenizer.convert_ids_to_tokens(tokens[i]) for i in range(tokens.shape[0])]
tokens_str = [[tokenizer.convert_tokens_to_string([t]) for t in t_list] for t_list in tokens_str]
# tokens_str = [[tokenizer.clean_up_tokenization(t) for t in t_list] for t_list in tokens_str]
tokens_str_len_arr = np.array([[len(t) for t in t_list] for t_list in tokens_str])
# get the np matrix of token_byte_pos
token_byte_pos = np.cumsum(tokens_str_len_arr, axis=1)

In [7]:
base_dir = output_dir + "/"

In [10]:
cb_at = "attn_preproj"

In [ ]:
import multiprocessing as mp
from functools import partial

# freq_filter = 5

def process_component(head=None, layer=None):
    # layer_dir = f'{output_dir}/layer{layer}'
    # os.makedirs(layer_dir, exist_ok=True)
    comp_name = f"layer{layer}_{cb_at}{ccb}{head if ccb else ''}"
    ft_tkns = features_to_tokens(comp_name, cb_acts, num_codes=num_codes)
    # with open(f"{layer_dir}/ft_tkns_layer{layer}{'_head'+head if ccb else ''}.pkl", "wb") as f:
    #     pickle.dump(ft_tkns, f)
    # with open(f"{layer_dir}/ft_tkns_layer{layer}{'_head'+head if ccb else ''}.pkl", "rb") as f:
    #     ft_tkns = pickle.load(f)
    # codes, freqs, acts = print_ft_tkns(ft_tkns,n=3,start=0,stop=10000,tokens=tokens_str,max_examples=100,freq_filter=freq_filter)
    # df = pd.DataFrame({"freqs": freqs, "acts": acts}, index=codes)
    # df.loc[:, "acts"] = df["acts"].apply(lambda x : x.replace('\n', '\\n'))
    # df.to_csv(f"{layer_dir}/code_token_acts_layer{layer}{'_head'+head if head else ''}_filt{freq_filter}.csv", index_label="code")
    acts_count = np.array([len(v) for v in ft_tkns])
    return f"layer{layer}{'_head'+str(head) if ccb else ''}", acts_count

act_count_ft_tkns = {}
if is_attn:
    with mp.Pool(min(16, cb_model.cfg.n_layers)) as pool:
        for head in range(0, cb_model.cfg.n_heads):
            process_component_partial = partial(process_component, head)
            for comp_name, acts_count in pool.map(process_component_partial, range(0, cb_model.cfg.n_layers)):
                act_count_ft_tkns[comp_name] = acts_count
else:
    for layer in tqdm(range(0, cb_model.cfg.n_layers)):
        k, v = process_component(layer=layer)
        act_count_ft_tkns[k] = v

    # with mp.Pool(16) as pool:
    #     process_component_partial = partial(process_component, None)
    #     for comp_name, acts_count in pool.map(process_component_partial, range(0, cb_model.cfg.n_layers)):
    #         act_count_ft_tkns[comp_name] = acts_count


In [ ]:
cb_acts.keys()

In [ ]:
from codebook_features.webapp import utils as webapp_utils
cache_path = ""
(
    tokens_str,
    tokens_text,
    token_byte_pos,
    cb_acts,
    act_count_ft_tkns,
    metrics,
) = webapp_utils.load_code_search_cache(cache_path)


In [ ]:
len(act_count_ft_tkns)

In [48]:
base_dir = "/shared/cb_eval_acts/tinystories-21m-1l_preproj_attn_2023-08-11_12-56-09/"


In [13]:
tokens.shape

(2000, 512)

In [16]:
ft_tkns = features_to_tokens("layer0_attn_preproj_ccb0", cb_acts, num_codes=10_000)

100%|██████████| 2000/2000 [00:02<00:00, 938.12it/s]


In [29]:
[b for a,b in ft_tkns[0]].index(985)

ValueError: 985 is not in list

In [28]:
ft_tkns[0][990]

(981, 267)

In [27]:
len(ft_tkns[0])

2023

In [12]:
# save all the data
np.save(base_dir + "tokens.npy", tokens)
np.save(base_dir + "tokens_str.npy", tokens_str)
np.save(base_dir + "tokens_text.npy", tokens_text)
np.save(base_dir + "token_byte_pos.npy", token_byte_pos)
# save act_count_ft_tkns using pickle
with open(base_dir + "act_count_ft_tkns.pkl", "wb") as f:
    pickle.dump(act_count_ft_tkns, f)

In [32]:
ft_tkns3 = features_to_tokens("layer0_attn_preproj_ccb3", cb_acts, num_codes=10_000)

100%|██████████| 2000/2000 [00:02<00:00, 908.59it/s]


In [39]:
ft_tkns = features_to_tokens("layer8_attn_preproj_ccb13", cb_acts, num_codes=10000)
ft_tkns = features_to_tokens("layer0_mlp", cb_acts, num_codes=10000)
# codes, freqs, acts = print_ft_tkns(ft_tkns, tokens=tokens_str, indices=[8518])

100%|██████████| 2000/2000 [00:57<00:00, 34.62it/s]


In [40]:
codes, freqs, acts = print_ft_tkns(ft_tkns, tokens=tokens_str, start=0, stop=100)

In [ ]:
for i, act in enumerate(acts):
    print(i)
    print(act[:20000])
    if i > 10:
        break

In [134]:
output_dir = "/shared/cb_eval_acts/pythia410_preproj_attn_2023-07-12_20-27-09/"

'/shared/cb_eval_acts/pythia410_preproj_attn*'

In [139]:
import pandas as pd
base_df_path = "/shared/cb_eval_acts/pythia410_preproj_attn_2023-07-12_20-27-09/"
layer, head = 7, 12
new_df = pd.read_csv(base_df_path + f"layer{layer}/" + f"code_token_acts_layer{layer}_head{head}_filt5.csv", index_col=0)

In [151]:
example_byte_pos = search_re("zini", tokens_text)
print(len(example_byte_pos))
print(example_byte_pos)
token_pos_ids = [byte_id_to_token_pos_id(example_byte_id, token_byte_pos) for example_byte_id in example_byte_pos]
print(token_pos_ids)
print([tokens_str_list[ex_id][t_id] for ex_id, t_id in token_pos_ids])

10
[(0, 0), (781, 3048), (781, 3071), (781, 3099), (781, 3124), (781, 3440), (781, 3545), (781, 3730), (781, 3805), (781, 4384)]
[(0, 0), (781, 661), (781, 670), (781, 679), (781, 688), (781, 767), (781, 791), (781, 836), (781, 861), (781, 1015)]
['z', 'azz', 'azz', 'azz', 'azz', 'azz', 'azz', 'azz', 'azz', 'azz']


In [7]:
print(metrics)

{'eval_loss': 2.293712854385376, 'eval_accuracy': 0.5102272727272728, 'eval_runtime': 7570.0973, 'eval_samples_per_second': 0.264, 'eval_steps_per_second': 0.008, 'eval_multicode_k': 1, 'eval_dead_code_fraction/layer0': 0.046875, 'eval_MSE/layer0': 108.40983496093747, 'eval_input_norm/layer0': 1.2060925477147104, 'eval_output_norm/layer0': 11.129539318561555, 'eval_dead_code_fraction/layer1': 0.0714625, 'eval_MSE/layer1': 112.84759632301332, 'eval_input_norm/layer1': 1.6721239459514616, 'eval_output_norm/layer1': 11.696583140373232, 'eval_dead_code_fraction/layer2': 0.09985625, 'eval_MSE/layer2': 65.05389386367797, 'eval_input_norm/layer2': 1.1916151970028874, 'eval_output_norm/layer2': 8.946532805204392, 'eval_dead_code_fraction/layer3': 0.00354375, 'eval_MSE/layer3': 122.12927466773989, 'eval_input_norm/layer3': 1.778145591557026, 'eval_output_norm/layer3': 12.187740631580352, 'eval_dead_code_fraction/layer4': 0.008625, 'eval_MSE/layer4': 80.80818650817871, 'eval_input_norm/layer4': 

In [77]:
cb_acts['layer23_attn_preproj_ccb15'].shape
tokens.shape

(2000, 1024)

In [79]:
output_dir = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = "/shared/cb_eval_acts/pythia410_preproj_attn_" + output_dir
os.makedirs(output_dir, exist_ok=True)
np.save(f"{output_dir}/tokens.npy", tokens)
np.save(f"{output_dir}/metrics.npy", metrics)
with open(f"{output_dir}/cb_acts.pkl", "wb") as f:
    pickle.dump(cb_acts, f)


In [ ]:
ccb = "_ccb13"
ft_tkns17_13 = features_to_tokens(f"layer17_{cb_at}{ccb}", cb_acts, num_codes=num_codes)


In [162]:
def print_dataset_example(dataset, example_idx=0):
    print(tokenizer.decode(dataset[example_idx]["input_ids"]))

In [174]:
print_dataset_example(dataset, example_idx=105)

 of 28 @,@ 952 at the 2001 census. The population density was 8 @,@ 425 inhabitants per square mile ( 3 @,@ 253 / km2 ), with a 100 – 92 @.@ 5 female @-@ to @-@ male ratio. Of those aged over 16, 19 @.@ 7 % were single ( never married ) and 41 % married. Cheadle Hulme's 11 @,@ 981 households included 27 @.@ 6 % one @-@ person, 43 @.@ 5 % married couples living together, 6 @.@ 5 % were co @-@ habiting couples, and 8 @.@ 1 % single parents with children. Of those aged 16 – 74, 20 @.@ 8 % had no academic qualifications. 
 About 76 @.@ 5 % of Cheadle Hulme's residents reported themselves as being Christian, 1 @.@ 9 % Muslim, 0 @.@ 9 % Jewish and 0 @.@ 7 % Hindu. The census recorded 12 @.@ 3 % as having no religion, 0 @.@ 3 % had an alternative religion and 7 @.@ 2 % did not state their religion. 
 = = Economy = = 
 For many years Cheadle Hulme was rural countryside, made up of woods, open land, and farms. The local population was made up of farmers and peasants, living in small cottages an

In [177]:
# separate string into tokens text
s = " Japan Japanese"
token_text = tokenizer.tokenize(s)
print(token_text)

['ĠJapan', 'ĠJapanese']


## Concept Code

In [3]:
cache_path = "/shared/cb_eval_acts/TinyStories-1Layer-21M#new-100ksteps_ccb_attn_preproj_2023-08-18_23-08-47/"
model_info = parse_model_info(cache_path)
num_codes, cb_at = model_info.num_codes, model_info.cb_at
tokens_str, tokens_text, token_byte_pos, cb_acts, act_count_ft_tkns, metrics = load_code_search_cache(cache_path)

In [4]:
ft_tkns = {}
concept_codes = {}

In [5]:
concept_codes_str = {}

In [47]:
concept = "scared"

In [46]:
cb_model.reset_hook_kwargs()
inp = " The kid was very scared as he was afraid of the monster."
gen = cb_model.generate(inp, prepend_bos=True, max_new_tokens=50, do_sample=True, temperature=1)
print(gen)
_, cache = cb_model.run_with_cache(gen)

  0%|          | 0/50 [00:00<?, ?it/s]

 The kid was very scared as he was afraid of the monster. He wished he could reach into it if he didn't leave it in the park. He hopped off and started crying.

Jimmy thought this for a minute and Then, he heard another voice: It said "I am here to rescue you!"


In [ ]:
cache['blocks.0.attn.codebook_layer.codebook.0.hook_codebook_ids'].shape

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.codebook_layer.codebook.0.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.1.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.2.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.3.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.4.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.5.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.6.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.7.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.8.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.9.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.10.hook_codebook_ids', 'blocks.0.attn.codebook_layer.codebook.

torch.Size([1, 64, 8])

In [42]:
cb_model.reset_hook_kwargs()
inp = ""
gen = cb_model.generate(inp, prepend_bos=True, max_new_tokens=50, do_sample=True, temperature=0.7)
print(gen)
_, cache = cb_model.run_with_cache(gen)

  0%|          | 0/50 [00:00<?, ?it/s]

 was safe.Once upon a time there was a girl called Lucy. She was very happy and grateful to the bird for helping her.

The bird flew around and sang a sweet song. It was the best picnic ever!

One day


In [ ]:
text_input = "Once upon a time"
inp_tensor = cb_model.to_tokens(text_input, prepend_bos=True).to(device)
inp_tensor = inp_tensor.repeat(10, 1)
list_of_arg_tuples = [
    # CodeInfo(layer=0, head=0, code=6, cb_at="attn", pos=None),
]
gen_patch = generate_with_codes(inp_tensor, cb_model, list_of_code_infos=list_of_arg_tuples, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":1})
output = [tokenizer.decode(gen_patch[i].cpu().numpy(), skip_special_tokens=True) for i in range(gen_patch.shape[0])]
for i in range(len(output)):
    print(output[i])
    print("**********************************\n")

In [ ]:
text_input = "Once upon a time"
inp_tensor = cb_model.to_tokens(text_input, prepend_bos=True).to(device)
inp_tensor = inp_tensor.repeat(10, 1)
pos = -1
# list_of_arg_tuples = [
#     CodeInfo(layer=0, head=14, code=723, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=14, code=4906, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=13, code=7800, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=13, code=8220, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=13, code=3861, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=13, code=11767, cb_at="attn", pos=-1),
#     CodeInfo(layer=0, head=11, code=4101, cb_at="attn", pos=-1),
# ]
# scared
# list_of_arg_tuples = [
#     CodeInfo(layer=0, head=11, code=15946, cb_at="attn", pos=pos),
#     CodeInfo(layer=0, head=11, code=15946, cb_at="attn", pos=pos),
# ]
# dragon
list_of_arg_tuples = [
    CodeInfo(layer=0, head=13, code=17640, cb_at="attn", pos=pos),
    CodeInfo(layer=0, head=13, code=19845, cb_at="attn", pos=pos),
    CodeInfo(layer=0, head=13, code=23958, cb_at="attn", pos=pos),
    CodeInfo(layer=0, head=13, code=3410, cb_at="attn", pos=pos),
    
]
gen_patch = generate_with_codes(inp_tensor, cb_model, list_of_code_infos=list_of_arg_tuples, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})
output = [tokenizer.decode(gen_patch[i].cpu().numpy(), skip_special_tokens=True) for i in range(gen_patch.shape[0])]
for i in range(len(output)):
    print(output[i])
    print("**********************************\n")

In [24]:
cb_model.reset_hook_kwargs()
inp = " One day, Lucy's mother"
gen = cb_model.generate(inp, prepend_bos=True, max_new_tokens=50, do_sample=True, temperature=0.7)
print(gen)
_, cache = cb_model.run_with_cache(gen)

  0%|          | 0/50 [00:00<?, ?it/s]

 One day, Lucy's mother had an idea. He asked her if they could go back to the store and buy the lamp.

One day, her mother told Lucy that she wanted to buy the lamp. Lucy was very excited and said yes. Her mother showed her how


In [185]:
# concept_codes_from = {}
concept_codes_from["football"] = gen
print(concept_codes_from)

# The movie was released by Columbia Pictures, which had bought the rights to produce the film in September 2007, and began production on the project in May 2008. The project's first cast member, Ty Burr, was cast as the title character, and was
# {'video_game': ' The game\'s designer, Maxime Imperi, commented that the game\'s development team " immediately recognized the limitations of the PlayStation ", and that they were able to integrate the features of the PlayStation\'s specifications into the game\'s engine.', 'football': " The football club's record attendance of 34 @,@ 049 in a UEFA Cup fifth round match against Juventus in November 1980. \n The following season, the club's record attendance of 36 @,@ 091 in an Intercontinental"}


{'video_game': ' The game\'s designer, Maxime Imperi, commented that the game\'s development team " immediately recognized the limitations of the PlayStation ", and that they were able to integrate the features of the PlayStation\'s specifications into the game\'s engine.', 'football': " The football club's record attendance of 34 @,@ 049 in a UEFA Cup fifth round match against Juventus in November 1980. \n The following season, the club's record attendance of 36 @,@ 091 in an Intercontinental"}


In [51]:
skips = 0
concept_codes_str[concept] = ""
for layer in range(cb_model.cfg.n_layers):
    for head in range(cb_model.cfg.n_heads):
        codes, occ_freq = common_codes_in_cache(cache[get_cb_layer_name("attn", layer, head)], threshold=75.0)
        if len(codes) == 0:
            continue
        
        if (layer, head) not in ft_tkns:
            ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
        _, freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=8,indices=codes,tokens=tokens_str,max_examples=50)
        for i in range(len(freq)):
            if freq[i][1] > 10:
                output = f"Skipping Code: {codes[i].item()}, Layer: {layer}, Head: {head}, Occ Freq: {occ_freq[i].item():.1f}, Train Act Freq: {freq[i]}"
                output = colored(output, "blue")
                print(output)
                skips += 1
                continue
            output = f"Code: {codes[i].item()}, Layer: {layer}, Head: {head}, Occ Freq: {occ_freq[i].item():.1f}, Train Act Freq: {freq[i][1]}"
            concept_codes_str[concept] += output + "\n"
            print(output)
            print(acts[i])


print(f"Skipped {skips} codes")

Code: 20642, Layer: 0, Head: 3, Occ Freq: 78.1, Train Act Freq: 0.19521484375
11:  the old woman.

The little girl worked
16: 
Suddenly, the man was filled with
18: 

The animal thought for a moment and then smiled. ��€�
20: 
They walked to the park, holding hands. Lily
23:  she would explore more tomorrow.Once upon a time, there was a little
25:  not, I saw it first."

Simba wouldn't budge.
26:  found a new friend and a place to explore
28:  They ran around the house and pretended to chase bad guys
36:  fixed the leak by putting a new washer
38: 
The police came and took Tom and Lily to the station
39: Tim was scared. He dropped the lamp. It
43:  She would often dream of things made from cocoa
44:  and kept the other half for herself. Ben took his half and walked over to Spot,
51:  would throw pillows at each other and laugh.

But one day, they heard a loud noise outside. It was a storm
53:  was.

Tommy explained that the poor dog was performing tricks for him. But the woman was not h

In [52]:
concept_codes_str = {}
concept_codes_str["video game"] = """
Code: 841, Layer: 3, Head: 7, Occ Freq: 95.8, Train Act Freq: 0.781591796875
Code: 4215, Layer: 3, Head: 7, Occ Freq: 93.8, Train Act Freq: 1.960498046875
Code: 6070, Layer: 3, Head: 7, Occ Freq: 91.7, Train Act Freq: 0.834130859375
Code: 8268, Layer: 3, Head: 7, Occ Freq: 91.7, Train Act Freq: 3.22880859375
Code: 1019, Layer: 13, Head: 6, Occ Freq: 81.2, Train Act Freq: 0.35302734375
Code: 9177, Layer: 16, Head: 15, Occ Freq: 89.6, Train Act Freq: 0.9982421875
Code: 2034, Layer: 16, Head: 15, Occ Freq: 81.2, Train Act Freq: 0.464306640625
Code: 3010, Layer: 18, Head: 10, Occ Freq: 93.8, Train Act Freq: 1.963427734375
Code: 1666, Layer: 18, Head: 10, Occ Freq: 91.7, Train Act Freq: 2.207177734375
Code: 868, Layer: 18, Head: 10, Occ Freq: 89.6, Train Act Freq: 2.29384765625
Code: 9545, Layer: 18, Head: 10, Occ Freq: 85.4, Train Act Freq: 1.716796875
Code: 3611, Layer: 18, Head: 10, Occ Freq: 85.4, Train Act Freq: 1.787255859375
"""

concept_codes_str["football"] = """
Code: 6182, Layer: 1, Head: 3, Occ Freq: 92.3, Train Act Freq: 2.213671875
Code: 9420, Layer: 1, Head: 3, Occ Freq: 82.7, Train Act Freq: 1.232666015625
Code: 8678, Layer: 3, Head: 7, Occ Freq: 94.2, Train Act Freq: 2.0642578125
Code: 8643, Layer: 3, Head: 7, Occ Freq: 94.2, Train Act Freq: 4.0583984375
Code: 5861, Layer: 3, Head: 7, Occ Freq: 94.2, Train Act Freq: 1.4658203125
Code: 1003, Layer: 3, Head: 7, Occ Freq: 92.3, Train Act Freq: 2.116064453125
Code: 4972, Layer: 3, Head: 7, Occ Freq: 88.5, Train Act Freq: 4.8748046875
Code: 4930, Layer: 3, Head: 7, Occ Freq: 82.7, Train Act Freq: 1.69892578125
Code: 4542, Layer: 3, Head: 7, Occ Freq: 82.7, Train Act Freq: 3.305859375
Code: 9965, Layer: 16, Head: 4, Occ Freq: 86.5, Train Act Freq: 1.748876953125
Code: 2258, Layer: 16, Head: 15, Occ Freq: 86.5, Train Act Freq: 1.1837890625
Code: 8484, Layer: 16, Head: 15, Occ Freq: 84.6, Train Act Freq: 1.151611328125
Code: 9736, Layer: 16, Head: 15, Occ Freq: 84.6, Train Act Freq: 0.86298828125
Code: 4376, Layer: 17, Head: 13, Occ Freq: 84.6, Train Act Freq: 1.37138671875
Code: 5048, Layer: 18, Head: 4, Occ Freq: 84.6, Train Act Freq: 1.1462890625
Code: 1729, Layer: 18, Head: 10, Occ Freq: 86.5, Train Act Freq: 1.953955078125
Code: 5488, Layer: 18, Head: 10, Occ Freq: 86.5, Train Act Freq: 3.181298828125
Code: 5813, Layer: 19, Head: 3, Occ Freq: 80.8, Train Act Freq: 0.6599609375
"""

# The movie
concept_codes_str["movie"] = """
Code: 9316, Layer: 3, Head: 7, Occ Freq: 96.2, Train Act Freq: 1.8212890625
Code: 2215, Layer: 3, Head: 7, Occ Freq: 92.5, Train Act Freq: 1.5474609375
Code: 5845, Layer: 3, Head: 7, Occ Freq: 83.0, Train Act Freq: 0.2470703125
Code: 9581, Layer: 16, Head: 15, Occ Freq: 83.0, Train Act Freq: 1.0712890625
Code: 237, Layer: 16, Head: 15, Occ Freq: 81.1, Train Act Freq: 0.814404296875
Code: 7256, Layer: 18, Head: 10, Occ Freq: 94.3, Train Act Freq: 3.094287109375
Code: 5883, Layer: 18, Head: 10, Occ Freq: 90.6, Train Act Freq: 1.87138671875
Code: 958, Layer: 18, Head: 10, Occ Freq: 88.7, Train Act Freq: 2.009033203125
Code: 9475, Layer: 18, Head: 10, Occ Freq: 88.7, Train Act Freq: 2.548828125
Code: 9899, Layer: 18, Head: 12, Occ Freq: 92.5, Train Act Freq: 1.791796875
Code: 9909, Layer: 18, Head: 12, Occ Freq: 84.9, Train Act Freq: 1.118212890625
Code: 4483, Layer: 20, Head: 1, Occ Freq: 81.1, Train Act Freq: 1.4267578125
"""
# The song
concept_codes_str["song"] = """
Code: 9932, Layer: 2, Head: 2, Occ Freq: 80.8, Train Act Freq: 0.249365234375
Code: 8605, Layer: 3, Head: 7, Occ Freq: 94.2, Train Act Freq: 3.56953125
Code: 1752, Layer: 3, Head: 7, Occ Freq: 80.8, Train Act Freq: 3.253466796875
Code: 2774, Layer: 3, Head: 7, Occ Freq: 90.4, Train Act Freq: 7.22744140625
Code: 5501, Layer: 3, Head: 7, Occ Freq: 86.5, Train Act Freq: 4.940966796875
Code: 405, Layer: 6, Head: 0, Occ Freq: 90.4, Train Act Freq: 1.462451171875
Code: 4344, Layer: 10, Head: 10, Occ Freq: 84.6, Train Act Freq: 0.62744140625
Code: 8090, Layer: 13, Head: 3, Occ Freq: 94.2, Train Act Freq: 1.57958984375
Code: 4764, Layer: 16, Head: 4, Occ Freq: 80.8, Train Act Freq: 2.605029296875
Code: 9154, Layer: 16, Head: 7, Occ Freq: 84.6, Train Act Freq: 0.710791015625
Code: 152, Layer: 16, Head: 7, Occ Freq: 80.8, Train Act Freq: 1.4203125
Code: 979, Layer: 17, Head: 13, Occ Freq: 94.2, Train Act Freq: 1.68388671875
Code: 9335, Layer: 17, Head: 13, Occ Freq: 94.2, Train Act Freq: 1.356884765625
Code: 559, Layer: 17, Head: 13, Occ Freq: 82.7, Train Act Freq: 0.837841796875
Code: 7647, Layer: 18, Head: 10, Occ Freq: 80.8, Train Act Freq: 1.2919921875
Code: 9420, Layer: 19, Head: 0, Occ Freq: 88.5, Train Act Freq: 6.741650390625
Code: 2034, Layer: 19, Head: 2, Occ Freq: 82.7, Train Act Freq: 1.614306640625
"""

In [52]:
concept_codes_str = {}
concept = "dragon"
concept_codes_str[concept] = """
Code: 4670, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 17640, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 19845, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 23958, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 3410, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 19523, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
Code: 2262, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
"""

concept = "Sam/Tom"
concept_codes_str[concept] = """
Code: 4848, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 17119, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 23081, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 23864, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 12184, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 10092, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
Code: 13973, Layer: 0, Head: 3, Occ Freq: -1, Train Act Freq: -1
"""

concept = "counting"
concept_codes_str[concept] = """
Code: 16089, Layer: 0, Head: 5, Occ Freq: -1, Train Act Freq: -1
Code: 1173, Layer: 0, Head: 10, Occ Freq: -1, Train Act Freq: -1
Code: 11261, Layer: 0, Head: 11, Occ Freq: -1, Train Act Freq: -1
Code: 5228, Layer: 0, Head: 11, Occ Freq: -1, Train Act Freq: -1
Code: 4169, Layer: 0, Head: 11, Occ Freq: -1, Train Act Freq: -1
Code: 2891, Layer: 0, Head: 11, Occ Freq: -1, Train Act Freq: -1
Code: 776, Layer: 0, Head: 13, Occ Freq: -1, Train Act Freq: -1
"""

concept = "slide"
concept_codes_str[concept] = """
Code: 1331, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 22178, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 15885, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 9524, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 15549, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 7802, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
Code: 11942, Layer: 0, Head: 14, Occ Freq: -1, Train Act Freq: -1
"""

In [175]:
prompts = [
    "The football club Real Madrid",
    "The war was fought",
    "The president of the United States",
    "The novel was published in",
    "The operating system was",
    "The team won",
    "The movie had a great",
    "The Road World Championships took place in Melbourne",
    "The main application of scandium by weight is in aluminium @-@ scandium alloys for",
    "The first single from the album",
    "The authority and position of the design management function has a large influence",
]

In [8]:
concept = "dragon"

In [56]:
pos = None
code_append = False
concept_codes = {k: parse_concept_codes_string(v, pos=pos, code_append=code_append) for k, v in concept_codes_str.items()}

list_of_arg_tuples = concept_codes[concept]
print(list_of_arg_tuples)

[CodeInfo(code=1331, layer=0, head=14, cb_at=attn, pos=None, code_pos=-1), CodeInfo(code=22178, layer=0, head=14, cb_at=attn, pos=None, code_pos=-2), CodeInfo(code=15885, layer=0, head=14, cb_at=attn, pos=None, code_pos=-3), CodeInfo(code=9524, layer=0, head=14, cb_at=attn, pos=None, code_pos=-4), CodeInfo(code=15549, layer=0, head=14, cb_at=attn, pos=None, code_pos=-5), CodeInfo(code=7802, layer=0, head=14, cb_at=attn, pos=None, code_pos=-6), CodeInfo(code=11942, layer=0, head=14, cb_at=attn, pos=None, code_pos=-7)]


In [58]:
text_input = "Once upon a time,"
inp_tensor = cb_model.to_tokens(text_input, prepend_bos=True).to(device)
inp_tensor = inp_tensor.repeat(10, 1)
# gen = generate_with_codes(text_input, cb_model, list_of_code_infos=[], generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})
gen = generate_with_codes(inp_tensor, cb_model, list_of_code_infos=list_of_arg_tuples, generate_kwargs={"max_new_tokens":51,"do_sample":True,"temperature":1})
gen = [tokenizer.decode(g[1:]) for g in gen]
for g in gen:
    print(g)
    print("************************************")

  0%|          | 0/51 [00:00<?, ?it/s]

Once upon a time, there was a big crab. The friend was a big, round fish, much bug and many other ducks in the pond. The frog liked to chase the ducks and play in the water. One day, a new idea came to the frog and said,
************************************
Once upon a time, there was a poor little girl named Lily.", Ben said, "We should all be careful when weâ€™re going on this holiday way again." 

Ben watched the planes take off and sat down. Lily and Sara pushed him off
************************************
Once upon a time, there was a green fox who lived in the forest. He liked pets and not want to play with them. One day, he saw something falling from the sky. He was curious and he wanted to play with it. 

Sam asked his Daddy
************************************
Once upon a time, there was a little boy named Tom. He was only three years old and loved to talk. One day, he was playing in his room when he saw something dark outside - a nail tooth on the ground. He had never seen an

In [16]:
list_of_arg_tuples[0].code_pos = 0
list_of_arg_tuples[1].code_pos = 1
list_of_arg_tuples[2].code_pos = 2
list_of_arg_tuples[3].code_pos = 3

In [67]:
text_input = "The team won"
inp_tensor = cb_model.to_tokens(text_input, prepend_bos=True).to(device)
inp_tensor = inp_tensor.repeat(10, 1)
# gen = generate_with_codes(text_input, cb_model, list_of_code_infos=[], generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})
gen = generate_with_codes(inp_tensor, cb_model, list_of_code_infos=list_of_arg_tuples, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})
gen = [tokenizer.decode(g[1:]) for g in gen]
for g in gen:
    print(g)

  0%|          | 0/50 [00:00<?, ?it/s]

The team won the 2013 Grammy Award for Best Gospel Album, Non @-@ Gospel Album, for The Holy Mountain. Subsequently, the group became the first band in its history to win three Grammy Awards, including Best Gospel Album. The group's third
The team won the Grammy for Best Rock Album at the 2011 Grammy Awards, which they also won for Best Rock Album and Best Rap / Sung Collaboration for The Blue Album. 
 The album debuted at number one in the UK for the week of December 2
The team won the award for Best Music Video at the 2010 Grammy Awards. 
 In March 2013, the song was included on the soundtrack album The Legends of Mimi and its accompanying EP Mimi : The Music Union Volume 1, which was released in April
The team won the right to sign with Gold Star Records. In an interview with MTV News, the group said " the album is definitely going to be a lot darker and darker in feeling and feeling ". During an interview with Billboard, the group explained, " We
The team won the state crown at the

In [ ]:
prompt_completions = {}
for concept in concept_codes:
    print(concept)
    list_of_arg_tuples = concept_codes[concept]
    prompt_completions[concept] = {}
    for prompt in prompts:
        prompt_token = cb_model.to_tokens(prompt, prepend_bos=True).to(device)
        prompt_token = prompt_token.repeat(10, 1)
        gen = generate_with_codes(
            prompt_token,
            cb_model,
            list_of_code_infos=list_of_arg_tuples,
            generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7,"num_return_sequences":10}
        )
        gen = [tokenizer.decode(gen[i][1:]) for i in range(len(gen))]
        prompt_completions[concept][prompt] = gen


In [226]:
print(prompt_completions["video game"]["The war was fought"][5])

The war was fought over. 
 In February 2016, TKOF announced that it had decided not to reveal the results of a 2015 E3 Rally Championship Series of the 2008 festival. 
 = = Reception = = 
 Since its inception in 1980, the R


In [227]:
import pandas as pd
def convert_prompt_complettions_to_csv(prompt_completions):
    df = pd.DataFrame(columns=["concept", "prompt", "completion"])
    for concept in prompt_completions:
        for prompt in prompt_completions[concept]:
            for completion in prompt_completions[concept][prompt]:
                df.loc[len(df)] = [concept, prompt, completion]

    print(len(df))
    df.to_csv("prompt_completions.csv", index=False)
    return df

df = convert_prompt_complettions_to_csv(prompt_completions)

440


In [190]:
cb_model.generate(prompt, prepend_bos=True, max_new_tokens=50, do_sample=True, temperature=0.7, num_return_sequences=10)

  0%|          | 0/50 [00:00<?, ?it/s]

"The president of the United States Olympic Committee. It is the world's most expensive atomic bomb and is the world's second @-@ heaviest atomic bomb. It is also the second @-@ heaviest particle accelerator, after the reactor @-@ like device used"

In [200]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model=orig_cb_model, tokenizer=cb_model.tokenizer, device=0)
generator(prompt, max_length=50, do_sample=True, temperature=0.7, num_return_sequences=10)

[ERROR|base.py:960] 2023-07-07 16:56:12,743 >> The model 'GPTNeoXCodebookModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCau

[{'generated_text': 'The president of the United States Navy, the Secretary of the Navy, and the Chairman of the Joint Chiefs of Staff, the commander of the Joint Chiefs of Staff, and Chairman of the Joint Chiefs of Staff ( <unk> ). \n = ='},
 {'generated_text': 'The president of the United States, Edward Teller, at the time of his death. \n = = Early life = = \n James T. " Bunny " Hays was born in the small town of North Carolina on 29 June 1901. He'},
 {'generated_text': "The president of the United States and Vice President of the United States, and the first woman to hold the inauguration of a president in office from the United States. She is the first woman to lead a nation's republic during a presidential election."},
 {'generated_text': 'The president of the United States of America in 1841. He resigned his seat in the U.S. Senate in 1845. He served as a member of the Connecticut delegation to the United States Congress in 1845, and was elected the 18'},
 {'generated_text': 'Th

In [ ]:
# The team won the tournament for the second straight season after beating Russia 3 – 1, Monten in the final round of the tournament. The win was followed by a 4 – 1 win over Japan in the final leg of the final. The Rangers defeated the United States 2 –

print_dataset_example(dataset, 24)

In [117]:
def suppress_codes_in_gen(input, code_info_tuples, cb_model, generate_kwargs=None):
    """Suppresses the codebook features in the generated text."""
    generate_kwargs = {} if generate_kwargs is None else generate_kwargs
    cb_model.reset_hook_kwargs()
    for code_info in code_info_tuples:
        attn_or_mlp = 0 if code_info.cb_at == "attn" else 1
        cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].set_hook_kwargs(keep_k_codes=False)
        # cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].hook_kwargs["disable_codes"].append(code_info.code)
        cb_model.all_codebooks[code_info.layer][attn_or_mlp].codebook[code_info.head].set_hook_kwargs(disable_topk=8,keep_k_codes=False)

    gen = cb_model.generate(input, **generate_kwargs)
    print(gen)
    return gen

text_input = "The album"
gen = suppress_codes_in_gen(text_input, list_of_arg_tuples, cb_model, generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})

  0%|          | 0/50 [00:00<?, ?it/s]

The album to be released on 12 October ( a day before the official date ), and on 30 August ( as a single ). 
 The album was released on CD, vinyl and Blu @-@ ray disc formats. The single for the album was released


In [111]:
len(set((cit.layer, ) for cit in list_of_arg_tuples))

12

In [104]:
cb_model.reset_hook_kwargs()
gen = generate_with_codes(text_input, cb_model, list_of_code_infos=[], generate_kwargs={"max_new_tokens":50,"do_sample":True,"temperature":0.7})


  0%|          | 0/50 [00:00<?, ?it/s]

Eminem, who noticed the resemblance to Sledge Shackleton's 1928 conquest of the " Incognito Mining School ". 
 = = = Critical reception = = = 
 " film reviews " was generally positive, with many critics praising the action


In [106]:
logits, cache = cb_model.run_with_cache("The album")
layer, head = 3, 7
ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
indices = cache[get_cb_layer_name("attn", layer, head)][0, -1]
print(indices)
# freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=3,indices=indices,tokenizer=tokenizer,tokens=tokens,max_examples=10)
# print(acts)

100%|██████████| 2000/2000 [00:03<00:00, 500.80it/s]


tensor([-1, -1, -1, -1, -1, -1, -1, -1], device='cuda:0')


In [ ]:
total_tokens = len(tokens) * len(tokens[0])
print(total_tokens)

In [ ]:
torch.unique(torch.randint(0, 10, (10, 10)), return_counts=True)

In [ ]:
print_ft_tkns(ft_tkns,n=5,start=0,stop=10,tokens=tokens_str,max_examples=100)

In [ ]:
seq = orig_cb_model.generate(do_sample=True, max_length=100, temperature=0.7)
tokenizer.decode(seq[0].tolist(), skip_special_tokens=True)

In [ ]:
cb_model.generate("The football club Manchester", max_new_tokens=5, do_sample=False)

In [ ]:
cb_model.model.cfg.device = device

In [ ]:
example_prompt = "Washington is the capital of The United"
example_answer = " States"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
example_prompt = "club Manchester"
example_answer = " United"

utils.test_prompt(
    example_prompt,
    example_answer,
    cb_model,
    prepend_bos=False,
    prepend_space_to_answer=False,
    top_k=10,
)

In [ ]:
input_a = "The football club Real"
input_b = "The football club Manchester"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")

logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
input_a = "Washington is the capital of The United"
input_b = "London is the capital of The United"

input_a = cb_model.to_tokens(input_a, prepend_bos=False).to("cuda")
input_b = cb_model.to_tokens(input_b, prepend_bos=False).to("cuda")

logits_a, cache_a = cb_model.run_with_cache(input_a)
logits_b, cache_b = cb_model.run_with_cache(input_b)

print(f"JSD(a, b) = {JSD(logits_a, logits_b)}")

In [ ]:
cache_a[get_cb_layer_name("attn", 17, 13)][0, -1]

In [ ]:
[0,17,19,20]

In [ ]:
# for layer_idx in range(cb_model.cfg.n_layers):
#     n_layers = 1
for layer_idx in range(1):
    # n_layers = cb_model.cfg.n_layers
    n_layers = 4
    cb_at = ( ["attn"] * cb_model.cfg.n_heads ) * n_layers
    head = (list(range(cb_model.cfg.n_heads))) * n_layers
    # layer = [layer_idx] * len(cb_at)
    # layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
    layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
    code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, -1, :] for i in range(len(cb_at))]
    # ind = [0,1,2,3,4,5]
    ind = range(len(cb_at))
    mod_logits, mod_cache = run_with_codes(
        input_a,
        cb_model,
        [code[i] for i in ind],
        [cb_at[i] for i in ind],
        [layer[i] for i in ind],
        [head[i] for i in ind],
        pos=[-1],
    )
    print("Layer:", layer_idx)
    print(logits_to_pred(mod_logits, tokenizer, k=5))
    print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")


In [ ]:
js_divs = torch.zeros((cb_model.cfg.n_layers, cb_model.cfg.n_heads))
for layer_idx in tqdm(range(cb_model.cfg.n_layers)):
    for head_idx in range(cb_model.cfg.n_heads):
        # n_layers = cb_model.cfg.n_layers
        cb_at = ["attn"]
        head = [head_idx]
        layer = [layer_idx] * len(cb_at)
        # layer = range_over_repeat(n_layers, repeat=cb_model.cfg.n_heads)
        # layer = range_over_repeat([0,17,19,20], repeat=cb_model.cfg.n_heads)
        code = [cache_b[get_cb_layer_name(cb_at[i], layer[i], head[i])][0, -1, :] for i in range(len(cb_at))]
        # ind = [0,1,2,3,4,5]
        ind = range(len(cb_at))
        mod_logits, mod_cache = run_with_codes(
            input_a,
            cb_model,
            [code[i] for i in ind],
            [cb_at[i] for i in ind],
            [layer[i] for i in ind],
            [head[i] for i in ind],
            pos=[-1],
        )
        # print("Layer, Head:", layer_idx, head_idx)
        # print(logits_to_pred(mod_logits, tokenizer, k=5))
        # print(f"JSD(a <- b, b) = {JSD(mod_logits, logits_b, pos=-1)}")
        js_divs[layer_idx, head_idx] = JSD(mod_logits, logits_b, pos=-1)

# make the color map range start from 0
imshow(js_divs, xaxis="head", yaxis="layer")



In [ ]:
imshow(js_divs, xaxis="head", yaxis="layer")

In [ ]:
js_divs.sort(axis=1)

In [ ]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    """Show an image."""
    px.imshow(
        tl_utils.to_numpy(tensor),
        color_continuous_scale="RdBu",
        labels={"x": xaxis, "y": yaxis},
        **kwargs,
    ).show(renderer)



In [ ]:
def range_over_repeat(end_or_list, repeat=1):
    
    if isinstance(end_or_list, int):
        end_or_list = range(end_or_list)
    l = []
    for i in end_or_list:
        l += [i] * repeat
    return l

## @-@ Code

In [239]:
inp = "The Blu"
ans = " @"
# ans = "@"
# ans = "-"
utils.test_prompt(
    inp,
    ans,
    cb_model,
    prepend_bos=True,
    prepend_space_to_answer=False,
    top_k=10,
)

Tokenized prompt: ['<|endoftext|>', 'The', ' Blu']
Tokenized answer: [' @']


Performance on answer token:
Rank: 2        Logit: 10.68 Prob:  6.24% Token: | @|

Top 0th token. Logit: 11.49 Prob: 13.93% Token: |estone|
Top 1th token. Logit: 10.81 Prob:  7.11% Token: | M|
Top 2th token. Logit: 10.68 Prob:  6.24% Token: | @|
Top 3th token. Logit: 10.50 Prob:  5.19% Token: | '|
Top 4th token. Logit: 10.30 Prob:  4.24% Token: | (|
Top 5th token. Logit: 10.14 Prob:  3.62% Token: | "|
Top 6th token. Logit: 10.11 Prob:  3.51% Token: | K|
Top 7th token. Logit: 10.08 Prob:  3.42% Token: | <|
Top 8th token. Logit:  9.54 Prob:  1.99% Token: |q|
Top 9th token. Logit:  9.49 Prob:  1.89% Token: | H|


Ranks of the answer tokens: [(' @', 2)]

In [240]:
layer, head = 3, 7
freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=5,start=0,stop=1000,tokens=tokens_str,max_examples=10)

In [241]:
for act in acts[:100]:
    print(act)

118:  which might be reflective of a ...  has longed for the starry throne where she has
200:  was found in the Miocene ...  a monstrous fluffy ginger beast ". 
 = =
208:  @-@ horned grasshoppers in
376:  appear reddish ) and the ...  is, four consecutive total eclipses with no partial eclips
470:  a peak rainfall amount of 3 @
652:  the Christmas Poo " and ...  Al's Big Gay Boat Ride ", which he felt included moral values ...  among the sophomoric humor. Stone also said those two episodes
703: . The popularity of his work ...  alone atop the <unk> hillfort. On 24 June
722:  had already been operated aboard several ...  Howard considers bringing Sheldon along to meet Hawking ...  and says that he has revolutionized understanding of the Higgs ...  stand too close to the urinal and urine often splashes back onto the buckles. Sheldon is given a black light and several dozen belt buckles to clean. Sheldon performs the task flawlessly. For his second task, Sheldon is forced to wear a French

In [ ]:
import re

def check_only_red_token(token_activation_str, red_token):
    """Returns True if all the red colored tokens in `token_activation_str` are `red_token`."""
    red_token_regex = red_token + "\x1b[0m"

    split_token_act = token_activation_str.split("\x1b[31m")[1:]
    if len(split_token_act) == 0:
        return False
    return all([tok.startswith(red_token_with_red_end) for tok in split_token_act])



red_token = " @"
for layer in range(1): # range(cb_model.cfg.n_layers):
    for head in range(cb_model.cfg.n_heads):
        if (layer, head) not in ft_tkns:
            ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
        freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=5,start=0,stop=10000,tokens=tokens_str,max_examples=10)
        for i, act in enumerate(acts):
            if check_only_red_token(act, red_token):
                print(f"Layer: {layer}, Head: {head}, Code: {i}, Freq: {freq[i]}")
                print(act)


In [ ]:
for act in acts_filt:
    print(act)

In [19]:
printed_ft_tkns = {}

In [ ]:
import multiprocessing
def print_ft_tkns(
    ft_tkns,
    tokens,
    tokenizer,
    separate_states=False,
    n=3,
    start=0,
    stop=1000,
    indices=None,
    max_examples=200,
    num_processes=None,
):
    """Prints the tokens for the codebook features using multiprocessing."""
    indices = list(range(start, stop)) if indices is None else indices
    num_tokens = len(tokens) * len(tokens[0])
    ft_tkns_indices = [ft_tkns[i] for i in indices]
    token_act_freqs = [100 * len(tkns) / num_tokens for tkns in ft_tkns_indices]

    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    # Create a pool of worker processes
    with multiprocessing.Pool(num_processes) as pool:

        def apply_tkn_print(ft_tkn):
            return tkn_print(
                ft_tkn,
                tokens,
                separate_states,
                n=n,
                max_examples=max_examples,
            )

        token_acts = pool.map(apply_tkn_print, ft_tkns_indices)

    return token_act_freqs, token_acts


In [21]:
def features_to_tokens(cb_key, cb_acts, num_codes):
    """Returns the set of token ids each codebook feature activates on."""
    codebook_ids = cb_acts[cb_key]
    features_tokens = [[] for _ in range(num_codes)]
    for i in tqdm(range(codebook_ids.shape[0])):
        for j in range(codebook_ids.shape[1]):
            for k in range(codebook_ids.shape[2]):
                features_tokens[codebook_ids[i, j, k]].append((i, j))

    return features_tokens


In [80]:
! du -hs $output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
47G	/shared/cb_eval_acts/pythia410_preproj_attn_2023-07-12_20-27-09


In [74]:
! ls /shared/cb_eval_acts/pythia410_preproj_attn_2023-07-11_05-35-16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
cb_acts.pkl  metrics.npy  tokens.npy


In [ ]:
from time import time
def search_pattern(token_activation_str, pattern):
    """Returns True if all the red colored tokens in `token_activation_str` are `red_token`."""
    pattern_regex = "\\x1b\[31m" + pattern + "\\x1b\[0m"
    matches = re.findall(pattern_regex, token_activation_str)
    return matches

# pattern = "\d"
# pattern = "(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|January|February|March|April|June|July|August|September|October|November|December|Mon|Tue|Wed|Thu|Fri|Sat|Sun|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)"
pattern = "(Playstation|Xbox|Nintendo|Sega|Atari)"
for layer in tqdm(range(cb_model.cfg.n_layers)):
    for head in range(cb_model.cfg.n_heads):
        if (layer, head) not in ft_tkns:
            ft_tkns[(layer, head)] = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
        # if (layer, head) not in printed_ft_tkns:
        freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=3,start=0,stop=10000,tokens=tokens_str,max_examples=100)
        # printed_ft_tkns[(layer, head)] = (freq, acts)
        df = pd.DataFrame({"freq": freq, "acts": acts})
        df.to_csv(f"code_token_acts_layer{layer}_head{head}.csv", index_label="code")
        # else:
        #     freq, acts = printed_ft_tkns[(layer, head)]
        # for i, act in enumerate(acts):
        #     if freq[i] > 2: # remove high freq codes
        #         continue
        #     matches = search_pattern(act, pattern)
        #     if len(matches) > 3:
        #         print(f"Layer: {layer}, Head: {head}, Code: {i}, Freq: {freq[i]}")
        #         print(act)


In [ ]:
import multiprocessing as mp
from functools import partial

def process_head(head, layer):
    layer_dir = f'{output_dir}/layer{layer}'
    os.makedirs(layer_dir, exist_ok=True)
    ft_tkns = features_to_tokens(f"layer{layer}_{cb_at}_ccb{head}", cb_acts, num_codes=num_codes)
    with open(f"{layer_dir}/ft_tkns_layer{layer}_head{head}.pkl", "wb") as f:
        pickle.dump(ft_tkns, f)
    codes, freqs, acts = print_ft_tkns(ft_tkns,n=3,start=0,stop=10000,tokens=tokens_str,max_examples=500,freq_filter=freq_filter)
    df = pd.DataFrame({"freqs": freqs, "acts": acts}, index=codes)
    df.loc[:, "acts"] = df["acts"].apply(lambda x : x.replace('\n', '\\n'))
    df.to_csv(f"{layer_dir}/code_token_acts_layer{layer}_head{head}_filt{freq_filter}.csv", index_label="code")

freq_filter = 5
with mp.Pool(16) as pool:
    for layer in tqdm(range(cb_model.cfg.n_layers)):
        print(f"Layer: {layer}")
        pool.map(partial(process_head, layer=layer), range(cb_model.cfg.n_heads))

In [137]:
tokens_str = []
for i, token in tqdm(enumerate(tokens)):
    tokens_str.append([tokenizer.decode(t) for t in token])
tokens_str = np.array(tokens_str)

2000it [00:08, 233.54it/s]


In [161]:
layer

23

In [136]:
tokenizer.decode(516)

' im'

In [ ]:
pattern = '(31m January|31m February|31m March|31m April|31m June|31m July|31m August|31m September|31m October|31m November|31m December)'

In [113]:
import csv
df.to_csv('temp_csv.csv', index_label="code")

In [111]:
df = df[df.freq <= 0.05]
df.loc[:, "acts"] = df["acts"].apply(lambda x : x.replace('\n', '\\n'))
len(df)

7883

In [84]:
import pandas as pd
df = pd.DataFrame({"freq": freq, "acts": acts})

In [91]:
df.to_csv("freq_acts.csv", index_label="code")

In [24]:
metrics

array({'eval_loss': 2.293712854385376, 'eval_accuracy': 0.5102272727272728, 'eval_runtime': 500.3652, 'eval_samples_per_second': 3.997, 'eval_steps_per_second': 0.126, 'eval_multicode_k': 1, 'eval_dead_code_fraction/layer0': 1.0, 'eval_MSE/layer0': 0.0, 'eval_input_norm/layer0': 0.0, 'eval_output_norm/layer0': 0.0, 'eval_dead_code_fraction/layer1': 1.0, 'eval_MSE/layer1': 0.0, 'eval_input_norm/layer1': 0.0, 'eval_output_norm/layer1': 0.0, 'eval_dead_code_fraction/layer2': 1.0, 'eval_MSE/layer2': 0.0, 'eval_input_norm/layer2': 0.0, 'eval_output_norm/layer2': 0.0, 'eval_dead_code_fraction/layer3': 1.0, 'eval_MSE/layer3': 0.0, 'eval_input_norm/layer3': 0.0, 'eval_output_norm/layer3': 0.0, 'eval_dead_code_fraction/layer4': 1.0, 'eval_MSE/layer4': 0.0, 'eval_input_norm/layer4': 0.0, 'eval_output_norm/layer4': 0.0, 'eval_dead_code_fraction/layer5': 1.0, 'eval_MSE/layer5': 0.0, 'eval_input_norm/layer5': 0.0, 'eval_output_norm/layer5': 0.0, 'eval_dead_code_fraction/layer6': 1.0, 'eval_MSE/laye

In [11]:
print(multiprocessing.cpu_count())

255


In [ ]:
Layer: 1, Head: 2, Code: 393, Freq: 0.0017578125
Layer: 1, Head: 4, Code: 6424, Freq: 0.02021484375
Layer: 1, Head: 4, Code: 7872, Freq: 0.00087890625
Layer: 1, Head: 4, Code: 3376, Freq: 0.001171875


Layer: 2, Head: 9, Code: 3106, Freq: 0.01630859375


Layer: 4, Head: 3, Code: 4716, Freq: 0.108935546875
Layer: 4, Head: 13, Code: 7713, Freq: 0.061962890625
Layer: 4, Head: 14, Code: 7287, Freq: 0.05703125


In [27]:
freq, acts = print_ft_tkns(ft_tkns[(layer, head)],n=2,start=0,stop=10000,tokens=tokens_str,max_examples=10)
for i, act in enumerate(acts):
    if freq > 2:
        continue
    print(act)

20:  line through South ...  plane before it crashed,
22:  - and Gregory ...  Conception of the Trinity = = = 
 Gregory, following Basil,
25:  former foster mother ...  a hill. He parked the ...  first day of shooting, director of
66: audouin, ...  off her arranged marriage with ...  war. Her tribe wins
143:  Lachlan ...  Wales with a rough map
188:  Baptist kneel
206:  with his second ... , it drew upon his
240:  the narrator never ...  a line in his ' O
299:  Ram Charan ...  in an interview that Magadheera could not
342:  Number 1 " ...  those she had composed on
358:  hearts, and ... ophis sirtalis
********************************************
11:  the Vuelta ...  final stage and won the
17:  of the season ... iquet in a BT50
27:  doesn 't ...  successfully orchestrates the escape of ...  is assigned to spearhead ...  his allegiance to the brothers
32:  crises in the ...  with France in 1337
35:  on the Japanese ...  the country achieved responsible government, with Sir Charles ... h

In [14]:
import sys
sys.getsizeof(ft_tkns[(0, 0)])

85176